## 수식의 표기법

### 중위 표기법
 
우리가 아는 수식 

```python
    (2 + 5) * 3 * (2 + 1)
```    

### 후위 표기법

컴퓨터가 계산하기 쉬운 수식으로 괄호가 없어도 우선순위가 보장된다. 그리고 연산자가 뒤로 간다.
(가정 : 연산식을 입력할 때는 한 자리 수의 연산으로 제한한다, 연산 도중에 2자리수의 연산은 처리 가능)

가정에 따라서, 연산자 앞에는 반드시 2개의 피연산자가 존재한다.

```python
    25+3*21+*
```

연산자는 스택에 쌓고 숫자는 리스트에 추가.
연산자의 가중치에 따라 넣는 방법이 달라진다. 

#### 가중치 비교

    `*`, `/` : 가장 높은 가중치
    `+`, `-` : 괄호보다 높은 가중치
    `(`, `)` : 가장 낮은 가중치

가중치가 낮은 것 위에 높은 연산자가 쌓이면 낮은 연산자 위에 그대로 쌓인다.
가중치가 같거나 낮은 연산자가 위에 쌓이면 이전 연산자는 후입선출식으로 리스트에 삽입된다.
`(`는 `)`를 만나면 바로 사라진다.

In [ ]:
class Stack(list):
    push = list.append       # 함수자체를 멤버변수로 받았음. 따라서 ()안붙임

# list는 이미 .pop를 내장함수로 지니고 있다. 
# push 랑 pop 선언은 이미 끝남. 

    def empty(self):
        if not len(self):    # if len(self) == 0: 와 같음
            return True
        else: 
            return False
    
    def peek(self):
        return self[-1]      # 맨 윗값 살펴보기


if __name__ == "__main__":
    s = Stack()
    s.push(1)
    s.push(2)
    s.push(3)
    s.push(4)
    s.push(5)
    
    while not s.is_empty(): # 비지 않았으면 계속 실행, 비었으면 break
        data = s.pop()
        print(data, end="  ")

In [60]:
class Calculator:
    def __init__(self):
        self.org_exp = None    # 중위표기법 수식을 받음
        self.postfix_exp = None    # 후위표기법 수식을 받음
        self.result = None
        
    
    # 파이썬의 setter    
    def set_org_exp(self, org_exp):
        self.org_exp = org_exp.replace(' ', '')
        
        self.postfix_exp = None    # 다시 초기화
        self.result = None
    
    
    # 파이썬의 getter
    def get_org_exp(self):
        return self.org_exp
    
    
    # 연산자의 가중치 설정
    def get_weight(self, oprt):
        if oprt == '*' or oprt == '/':
            return 9
        elif oprt == '+' or oprt == '-':
            return 7
        elif oprt == '(':
            return 5
        else:
            return -1
    
    
    # 후위표기법 계산
    def convert_to_postfix(self):
        exp_list = []
        oprt_stack = Stack()
        
        for ch in self.get_org_exp():
            if ch.isdigit():    # 숫자 문자열인지 확인
                exp_list.append(ch)
            # 연산자가 올 때
            else:    
                # oprt_stack이 비었거나 '('가 올 경우
                if ch == '(' or oprt_stack.empty():
                    oprt_stack.push(ch)
                elif ch == ')':
                    op = oprt_stack.pop()
                    while op != '(':    # ')'가 '('를 만날 때까지 while문 실행. 
                        exp_list.append(op)    # 리스트에 pop으로 뽑은 데이터를 집어넣는다.    
                        op = oprt_stack.pop()
                
                # +, -, *, / 다음에 쌓일 연산자의 가중치가 높은 경우
                elif self.get_weight(ch) > self.get_weight(oprt_stack.peek()):
                    oprt_stack.push(ch)
                
                # 다음에 쌓일 연산자의 가중치가 같거나 낮을 경우 
                else:
                    # oprt_stack이 비어있으면 pop()에서 에러가 나므로 먼저 확인해준다.
                    while oprt_stack and (self.get_weight(ch) <= self.get_weight(oprt_stack.peek())):
                        exp_list.append(oprt_stack.pop())
#                     oprt_stack.append(ch)
                    oprt_stack.push(ch)
        
        while oprt_stack:
            exp_list.append(oprt_stack.pop())
        
        # 저장
        self.postfix_exp = ''.join(exp_list)
         
                        
    # 후위표기법 계산된 식을 요청할 때 캐싱된 값을 반환(연산이 된 경우)
    # 연산이 안된 경우에는 후위표기법 계산식을 리턴한다.
    def get_postfix_exp(self):
        if not self.postfix_exp:
            self.convert_to_postfix()
        
        return self.postfix_exp

    
    # 연산자에 따른 리턴값 설정
    def calc_two_oprd(self, oprd1, oprd2, oprt):
        if oprt == '+':
            return oprd1 + oprd2
        elif oprt == '-':
            return oprd1 - oprd2
        elif oprt == '*':
            return oprd1 * oprd2
        elif oprt == '/':
            return oprd1 // oprd2
        
        
    def calculate(self):
        # oprd : 피연산자(숫자)
        # oprt : 연산자
        oprd_stack = Stack()
        
        # 클래스외부에서는 액세스 함수를 정의했다면 해당 액세스 함수를 통해 접근해야한다.
        # 클래스 내부에서도 액세스 함수가 있다면 이를 통해 접근하는 게 좋다.
        for ch in self.get_postfix_exp():
        # for ch in self.postfix_exp: 와 동일
            if ch.isdigit():
                # 숫자형으로 스택에 넣으므로 연산된 값은 자리수에 상관없이 저장된다.
                oprd_stack.push(int(ch))
            else:
                # -, / 연산은 피연산자 순서가 바뀌면 안되므로 변수이름을 유의하여 설정해준다.
                oprd2 = oprd_stack.pop()
                oprd1 = oprd_stack.pop()
                oprd_stack.push(self.calc_two_oprd(oprd1, oprd2, ch))
    
        self.result = oprd_stack.pop()
        
    # 결과값 출력 - 계산이 된 경우에는 캐싱된 값 사용 / 아니라면 calculate() 메서드 사용
    def get_result(self):
        if not self.result:
            self.calculate()
            
        return self.result

### 실행

In [ ]:
calc = Calculator()
while 1:
    exp = input("수식을 입력하세요(종료 : 0) : ")
    if exp == '0':
        break

    calc.set_org_exp(exp)
    postfix = calc.get_postfix_exp()
    print('postfix_exp: ', postfix)

    print('{exp} = {result}'.format(
        exp = calc.get_org_exp(),
        result = calc.get_result()))

수식을 입력하세요(종료 : 0) : 1 + 2 *3
postfix_exp:  123*+
1+2*3 = 7


In [ ]:
### 선생님 풀이

# from stack import Stack

class Calculator:
    def __init__(self):
        self.org_exp = None
        self.postfix_exp = None
        self.result = None

    def set_org_exp(self, org_exp):
        self.org_exp = org_exp.replace(' ', '')
        self.postfix_exp = None
        self.result = None

    def get_org_exp(self):
        return self.org_exp
    
    def get_weight(self, oprt):
        if oprt == '*' or oprt == '/':
            return 9
        elif oprt == '+' or oprt == '-':
            return 7
        elif oprt == '(':
            return 5
        else:
            return -1

    def convert_to_postfix(self):
        exp_list = []
        oprt_stack = Stack()
        
        for ch in self.org_exp:
            if ch.isdigit():
                exp_list.append(ch)
            #연산자라면
            else:
                #'(' or 연산자 스택이 비었다면
                if ch == '(' or oprt_stack.empty():
                    oprt_stack.push(ch)
                #')'
                elif ch == ')':
                    op = oprt_stack.pop()
                    while op != '(':
                        exp_list.append(op)
                        op = oprt_stack.pop()
                #'+', '-', '*', '/' : 가중치가 높을 때
                elif self.get_weight(ch) > \
                    self.get_weight(oprt_stack.peek()):
                    oprt_stack.push(ch)
                #'+', '-', '*', '/' : 가중치가 낮거나 같을 때
                else:
                    while oprt_stack and self.get_weight(ch) <=\
                          self.get_weight(oprt_stack.peek()):
                        exp_list.append(oprt_stack.pop())
                    oprt_stack.push(ch)
        while oprt_stack:
            exp_list.append(oprt_stack.pop())
        self.postfix_exp = ''.join(exp_list)

    def get_postfix_exp(self):
        if not self.postfix_exp:
            self.convert_to_postfix()

        return self.postfix_exp

    def calc_two_oprd(self, oprd1, oprd2, oprt):
        if oprt == '+':
            return oprd1 + oprd2
        elif oprt == '-':
            return oprd1 - oprd2
        elif oprt == '*':
            return oprd1 * oprd2
        elif oprt == '/':
            return oprd1 // oprd2

    def calculate(self):
        oprd_stack = Stack()

        for ch in self.postfix_exp:
            if ch.isdigit():
                oprd_stack.push(int(ch))
            #연산자라면
            else:
                oprd2 = oprd_stack.pop()
                oprd1 = oprd_stack.pop()
                oprd_stack.push(self.calc_two_oprd(
                    oprd1, oprd2, ch))
        self.result = oprd_stack.pop()

    def get_result(self):
        if not self.result:
            self.calculate()
            
        return self.result

if __name__ == "__main__":
    calc = Calculator()
    while 1:
        exp = input("수식을 입력하세요(종료 : 0) : ")
        if exp == '0':
            break
        
        calc.set_org_exp(exp)
        print(calc.get_postfix_exp())
        print("{exp} = {result}".format(
            exp = calc.get_org_exp(), result = calc.get_result()))